In [16]:
import os
import cv2
import time
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

from my_funcs.plot_functions import mask_roi_finder

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction


## 1.1. Enter Data: ##

In [17]:
phantom_choice = 1

# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_phantom_fns = [os.path.join(general_fn, 'scans', '23_12_08_glu_phantom_37deg',
                                '1_glu_phantom_12_16_20mM_ph7_37deg'),
                   os.path.join(general_fn, 'scans', '23_12_08_glu_phantom_37deg',
                                '2_glu_phantom_12mM_ph6p5_7_7p4_37deg')]  # December 23
glu_phantom_fn = glu_phantom_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'


## 1.2. scan data ##

In [18]:
# fix all those!
conc_ls = [[20, 12, 16],
           [12, 12, 12]]

ph_ls = [['pH 7', 'pH 7', 'pH 7'],
         ['pH 7', 'pH 7.4', 'pH 6.5']]

tags = [['a', 'c', 'b'], 
        ['b', 'a', 'c']]

tag_x_locs = [[-3,3,-2],
              [-2,1,-2]]

tag_y_locs = [[-13,13,14],
              [-13,-13,12]]

cest_prtcls_names = [c,
                     ['EPI_0p7uT', 'EPI_2uT', 'EPI_4uT', 'EPI_6uT']]

temperatures = ['37°C', '37°C']
months = ['dec', 'dec']

B_names = [['0.7uT', '2uT', '4uT', '6uT'], ['0.7uT', '2uT', '4uT', '6uT']]


## 1. z-spectra ##

In [19]:
mean_z_spectra = []
mean_z_spectra_n_c = []
gyro_ratio_hz = 42.5764  # for H [Hz/uT]
b0 = 7

for phantom_i in [0, 1]:
    cest_prtcl_names = cest_prtcls_names[phantom_i]
    glu_phantom_fn = glu_phantom_fns[phantom_i]
    B_name = B_names[phantom_i]
    
    # mask
    _, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
    vial_rois, full_mask, _ = mask_roi_finder(bruker_dataset_mask)
    
    # WASSR image
    wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, 'WASSR')
    wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
    M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)

    wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
    offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    b0_map = wassr_b0_mapping(wassr_norm, full_mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz*b0)
    
    mean_z_spectrum = np.zeros([3, len(cest_prtcl_names), 57])
    mean_z_spectrum_not_cor = np.zeros([3, len(cest_prtcl_names), 57])
    for cest_prtcl_i, cest_prtcl_name in enumerate(cest_prtcl_names):  # 0.7, 2, 4, 6
        cur_B_name = B_name[cest_prtcl_i]

        # z-spec
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, cest_prtcl_name)  # (58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
        cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
        M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
        z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
        
        # a = z_spec_norm[0,:,:]*(~full_mask - 254)
        # a_1 = z_spec_norm[0,:,:]*(full_mask)
        
        # offset vector
        offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
        offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    
        b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz) # have not checked!

        for vial_i, vial_roi in enumerate(vial_rois):
            roi_mask = np.zeros_like(full_mask)
            rr, cc = vial_roi.coords[:, 0], vial_roi.coords[:, 1]
            roi_mask[rr, cc] = 1
            # roi_mask = cv2.cvtColor(roi_mask, cv2.COLOR_BGR2GRAY)
            roi_mask = roi_mask.astype(np.uint8)
            
            for c_i in range(z_spec_norm.shape[0]):
                [[mean_vial]], _ = cv2.meanStdDev(z_spec_norm[c_i, :, :], mask=roi_mask)  # before b0 correction
                mean_z_spectrum_not_cor[vial_i,cest_prtcl_i,c_i] = mean_vial
                [[mean_vial]], _ = cv2.meanStdDev(b0_cor_zspec[c_i, :, :], mask=roi_mask)
                mean_z_spectrum[vial_i,cest_prtcl_i,c_i] = mean_vial

    mean_z_spectra.append(mean_z_spectrum)
    mean_z_spectra_n_c.append(mean_z_spectrum_not_cor)
    

WASSR B0 mapping took 3.354 seconds
WASSR B0 mapping took 3.847 seconds


In [24]:
from lmfit import Model
import numpy as np

def z_cw_creator(w_df,  # ppm
                 b1_v,
                 zi_v,
                 kb, t1w, t2w,
                 tp, fb, t1s, t2s,
                 gyro_ratio_hz = 42.5764,  # gamma for H [Hz/uT]
                 b0 = 7):
    gamma = 267.5153  # [MHz]
    
    # Define the function as a lambda expression
    rex_lorentz = lambda da, w1, db, fb, kb, r2b, zi: (
        (
            (kb * fb) * w1**2 / (da**2 + w1**2) *
            ((da - db)**2 + (da**2 + w1**2) * r2b / kb + r2b * (kb + r2b))
        ) / (
            ((2 * np.sqrt((kb + r2b) / kb * w1**2 + (kb + r2b)**2)) / 2)**2 + db**2
        )
    )
    r1w = 1/t1w
    r2w = 1/t2w
    r1s = 1/t1s
    r2s = 1/t2s
    z_cw = np.array([])  # (4, 57)
    for b1_i, b1 in enumerate(b1_v):
        zi = zi_v[b1_i]
        w1 = b1 * gamma
        w_ref = b0 * gyro_ratio_hz * 2 * np.pi
        da = (w_df - 0) * w_ref  # 0 ppm water
        db = (w_df - 3) * w_ref  # 3 ppm solute
        
        w1 = np.full(len(w_df), w1)
        da = np.where(da == 0, 1e-8, da)
        theta = np.arctan(w1 / da)
        
        rex = rex_lorentz(da,w1,db,fb,kb,r2s,zi)  # exchange dependant relaxation (rotating frame)
        reff = r1w * np.cos(theta)**2 + r2w * np.sin(theta)**2  # R1rho of water
        r1rho = reff+rex
        pz = np.cos(theta)
        pzeff = np.cos(theta)
        r1rho = np.where(r1rho == 0, 1e-8, r1rho)
        zss = np.cos(theta) * pz * r1w / r1rho
        cur_z_cw = (pz * pzeff * zi - zss) * np.exp(-(r1rho*tp)) + zss
        z_cw = np.concatenate((z_cw, cur_z_cw))
        
    return z_cw


def multi_b1_fit(w_df,
                 b1_v,
                 zi_v,
                 z_spec_mat,  # (vial i, b1 i, 7 to -7)
                 params):
    # z_cw = z_cw_creator(params, b1_v, w_df)
    
    # I will try varying kb, r1w, r2w (maybe r1s, r2s next?)
    piecewisez_cw = lambda w_df, kb, t1w, t2w, tp, fb, t1s, t2s: (
        z_cw_creator(w_df, b1_v, zi_v, kb, t1w, t2w, tp, fb, t1s, t2s))

    # u = piecewisez_cw(w_df, 7000, 1/4, 1/1.8, 3, 0.01, 1/1, 1/0.04)
    
    ft = Model(piecewisez_cw)
            # Add bounds to parameters
    opts = ft.make_params(method='leastsq')  # Method for optimization
    
    # varying
    opts.add('kb', value=7500, min=2000, max=12000)
    opts.add('t1w', value=params['t1w'], min=2.5, max=4.5)
    opts.add('t2w', value=params['t2w'], min=0.5, max=1.9)
    opts.add('t1s', value=params['t1s'], min=1, max=4.5)
    opts.add('t2s', value=params['t2s'], min=0.007, max=0.2)
    
    # constant
    opts.add('t1w', value=params['t1w'], vary=False)
    opts.add('t2w', value=params['t2w'], vary=False)
    opts.add('tp', value=params['tp'], vary=False)
    opts.add('fb', value=params['fb'], vary=False)
    # opts.add('t1s', value=params['t1s'], vary=False)
    # opts.add('t2s', value=params['t2s'], vary=False)
    
    # b1s = np.array([])
    zs = np.array([])
    for b1_i, b1 in enumerate(b1_v):
        # b1s = np.concatenate((b1s, np.full(len(w_df), b1)))
        zs = np.concatenate((zs, z_spec_mat[b1_i, :]))  # 7 to -7
    
    # Perform the curve fitting
    result = ft.fit(zs, w_df=w_df,
                    kb=opts['kb'], t1w=opts['t1w'], t2w=opts['t2w'],
                    tp=opts['tp'], fb=opts['fb'], t1s=opts['t1s'], t2s=opts['t2s'],
                    method='least_squares')
              
    # # Extract fit results and goodness-of-fit
    # fitresult = result.best_values
    # gof = result.fit_report(result.params)
    # ci = lmfit.conf_interval(result, result, sigmas=[2])  # not exact same 2 sigma instead of 95%
    # kb_ci = ci['kb'][2][1] - np.mean([ci['kb'][2][1], ci['kb'][0][1]])  # 95%
    # fb_ci = ci['fb'][2][1] - np.mean([ci['fb'][2][1], ci['fb'][0][1]])  # 95%
    fitted_z = result.best_fit
    
    return result, fitted_z


In [25]:
b1_i_lim = [2, 4]

b1_v = [0.7, 2, 4, 6][b1_i_lim[0]:b1_i_lim[1]]
zi_v = mean_z_spectra[0][0, b1_i_lim[0]:b1_i_lim[1], -1]
# zi_v = np.ones((4))
w_df = np.arange(7, -7.25, -0.25)

phantom_i = 0  # dec

tag = tags[phantom_i]
vial_i = 0
tag_l = [0, 2, 1]
tag_i = tag_l[vial_i]

cest_prtcl_names = cest_prtcls_names[phantom_i][b1_i_lim[0]:b1_i_lim[1]]  # 0.7, 2, 4, 6
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
conc_l = conc_ls[phantom_i]
ph_l = ph_ls[phantom_i]
tag = tags[phantom_i]
month = months[phantom_i]

# Iterate over each image
for tag_i, cur_tag in enumerate(['a', 'b', 'c']):
    vial_i = tag.index(cur_tag)
    params = {
    'tp': 3,  #[s]
    'fb': conc_l[vial_i]*3/110000,
    'kb': 7500,
    't1w': 4,
    't2w': 1.8,
    't1s': 2,
    't2s': 0.05
    }
    fig = make_subplots(rows=1, cols=len(cest_prtcl_names),
                        subplot_titles=B_names[phantom_i][b1_i_lim[0]:b1_i_lim[1]], vertical_spacing = 0.07, horizontal_spacing=0.01)
    
    z_spec_mat = mean_z_spectra[phantom_i][vial_i, b1_i_lim[0]:b1_i_lim[1], :]
    fit_result, fitted_z = multi_b1_fit(w_df, b1_v, zi_v, z_spec_mat, params) 
    fitted_z_rs = fitted_z.reshape((len(b1_v), 57))
    
    for b1_i, b1 in enumerate(b1_v):
        col_idx = b1_i + 1
        
        cur_z = z_spec_mat[b1_i, :]
        cur_z_fit = fitted_z_rs[b1_i, :]
        fig.add_trace(go.Scatter(x=w_df, y=cur_z, line=dict(color=colors[tag_i]), name=f'real z', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
        fig.add_trace(go.Scatter(x=w_df, y=cur_z_fit, line=dict(color=colors[tag_i]), opacity=0.5, name=f'fit z', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
    
    fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
    
    # Update layout
    fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                      title_text=f'Phantom {month} - {temperatures[phantom_i]} Z-spectrum - {conc_l[vial_i]}mM, {ph_l[vial_i]}',
                      height=300, width=350*len(cest_prtcl_names),
                      title=dict(x=0.02, y=0.97),
                      margin=dict(l=45, r=0, t=50, b=0)
                      )  # Adjust the title position
    
    
    # Set axes for z-spectrum
    fig.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1.15])
    
    # Show only specific legend groups
    for trace in fig.data:
        trace.showlegend = (trace.legendgroup == '1_1_1')
    
    fig.show()
    
    # After fitting the model
    print(fit_result.fit_report())  # Print a summary of the fit results
    # print("Parameter Values:")
    # for name, param in fit_result.params.items():
    #     if name == 'fb':
    #         print(f"{name}: {param.value * 110000 / 3:.2f} +/- {param.stderr * 110000 / 3:.2f}")
    #     else:
    #         print(f"{name}: {param.value:.2f} +/- {param.stderr:.2f}")
    
    pio.write_image(fig, f'images/z_spec/z_spec_dec_{conc_l[vial_i]}mM_fit.jpeg')


[[Model]]
    Model(<lambda>)
[[Fit Statistics]]
    # fitting method   = least_squares
    # function evals   = 1020
    # data points      = 114
    # variables        = 3
    chi-square         = 0.02700945
    reduced chi-square = 2.4333e-04
    Akaike info crit   = -945.645413
    Bayesian info crit = -937.436818
    R-squared          = 0.99712566
##  Warning: uncertainties could not be estimated:
    t2s:  at boundary
[[Variables]]
    kb:   8145.71713 (init = 7500)
    t1w:  4 (fixed)
    t2w:  1.8 (fixed)
    tp:   3 (fixed)
    fb:   0.0005454545 (fixed)
    t1s:  1.00178492 (init = 2)
    t2s:  0.00700000 (init = 0.05)


[[Model]]
    Model(<lambda>)
[[Fit Statistics]]
    # fitting method   = least_squares
    # function evals   = 1116
    # data points      = 114
    # variables        = 3
    chi-square         = 0.02291544
    reduced chi-square = 2.0645e-04
    Akaike info crit   = -964.384262
    Bayesian info crit = -956.175667
    R-squared          = 0.99749458
##  Warning: uncertainties could not be estimated:
    t2s:  at boundary
[[Variables]]
    kb:   7954.13006 (init = 7500)
    t1w:  4 (fixed)
    t2w:  1.8 (fixed)
    tp:   3 (fixed)
    fb:   0.0004363636 (fixed)
    t1s:  1.00145070 (init = 2)
    t2s:  0.00700000 (init = 0.05)


[[Model]]
    Model(<lambda>)
[[Fit Statistics]]
    # fitting method   = least_squares
    # function evals   = 28
    # data points      = 114
    # variables        = 3
    chi-square         = 0.02170904
    reduced chi-square = 1.9558e-04
    Akaike info crit   = -970.549638
    Bayesian info crit = -962.341042
    R-squared          = 0.99745299
##  Warning: uncertainties could not be estimated:
    t2s:  at boundary
[[Variables]]
    kb:   9615.84960 (init = 7500)
    t1w:  4 (fixed)
    t2w:  1.8 (fixed)
    tp:   3 (fixed)
    fb:   0.0003272727 (fixed)
    t1s:  4.48366134 (init = 2)
    t2s:  0.00700000 (init = 0.05)
